## Preprocessing

In [15]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
!pip install keras-tuner --upgrade

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [16]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'],inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [18]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts =application_df['APPLICATION_TYPE'].value_counts()
type(app_type_counts)
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace` - replacing 9 values
# ITERATION 1 - improve the distribution of bins across APPLICATION_TYPE by combining those below 700, thus increasing values in the lower bins. This did not increase the accuracy significantly (73%).
application_types_to_replace = app_type_counts.loc[app_type_counts<700].index
application_types_to_replace
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Look at CLASSIFICATION value counts for binning
classification_counts =application_df['CLASSIFICATION'].value_counts()
type(classification_counts)
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [21]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_copy=classification_counts.copy()
classification_counts_copy.loc[classification_counts_copy>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace` - counts less than 700
# ITERATION 2 - improve the distribution of bins across classifications by combining those below 700, thus increasing values in the lower bins. This did not increase the accuracy significantly (73%).
classifications_to_replace = classification_counts.loc[classification_counts<700].index
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_final=pd.get_dummies(application_df)
application_df_final.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [24]:
from numpy.random.mtrand import random_integers
# Split our preprocessed data into our features and target arrays
y = application_df_final['IS_SUCCESSFUL'].values
X = application_df_final.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,stratify=y)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled
X_test_scaled

array([[ 0.01079982, -0.03253341, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.03253341, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.03253341, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687],
       ...,
       [ 0.01079982, -0.03253341, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.03253341, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.03228971, -0.15558816, ..., -0.07450354,
         0.02925687, -0.02925687]])

## Compile, Train and Evaluate the Model

In [26]:
# Modified and iterarted this section of code multiple times for the following:
# * Activation for hidden layers - tried with relu first and added tanh
# * Nuerons min and max value was modified to take min value from first model ie. 16
# * Number of hidden layers increased to 3 from first model

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide from 40 to 100 neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=40,max_value=100,step=5),
                                       activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide from 1 to 3 hidden layers and 40 to 100 neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=40,
            max_value=100,
            step=5),
            activation=activation))

    #Final Layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [27]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [28]:
 # Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=30,validation_data=(X_test_scaled,y_test))

Trial 247 Complete [00h 03m 23s]
val_accuracy: 0.7276967763900757

Best val_accuracy So Far: 0.7280466556549072
Total elapsed time: 01h 32m 53s

Search: Running Trial #248

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
36                |32                |first_units
1                 |1                 |num_layers
24                |32                |units_0
16                |24                |units_1
100               |12                |tuner/epochs
34                |0                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |0                 |tuner/round
0243              |None              |tuner/trial_id

Epoch 35/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5379 - accuracy: 0.7392 - val_loss: 0.5531 - val_accuracy: 0.7241
Epoch 36/100
129/804 [===>..........................] - ETA: 1s - loss: 0.5401 - accuracy: 0.7359

KeyboardInterrupt: ignored

In [29]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 32, 'num_layers': 1, 'units_0': 32, 'units_1': 24, 'tuner/epochs': 12, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 40, 'num_layers': 1, 'units_0': 36, 'units_1': 20, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0203'}
{'activation': 'relu', 'first_units': 24, 'num_layers': 1, 'units_0': 20, 'units_1': 24, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0233'}


In [36]:
top_model = tuner.get_best_models(1)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    model.save('AlphabetSoupCharity_Optimized.h5')

268/268 - 1s - loss: 0.5556 - accuracy: 0.7280 - 578ms/epoch - 2ms/step
Loss: 0.555617094039917, Accuracy: 0.7280466556549072


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'tanh',
 'first_units': 40,
 'num_layers': 1,
 'units_0': 36,
 'units_1': 20,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 34,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0203'}

In [38]:
# Compare the performance to the second-best model
second_model = tuner.get_best_models(2)[1]
model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5519 - accuracy: 0.7279 - 556ms/epoch - 2ms/step
Loss: 0.5518989562988281, Accuracy: 0.7279300093650818
